In [1]:
import numpy as np
from ompl import base as ob
from ompl import geometric as og
from pydrake.planning import RobotDiagramBuilder
from pydrake.math import RigidTransform
from airo_drake import add_meshcat, finish_build, visualize_frame
import airo_models
from dataclasses import dataclass
from ConstrainedPlanningCommon import ConstrainedProblem

In [2]:
# Create the robot diagram builder
robot_diagram_builder = RobotDiagramBuilder()

# Add meshcat visualizer
meshcat = add_meshcat(robot_diagram_builder)

# Get plant and parser from the builder
plant = robot_diagram_builder.plant()
parser = robot_diagram_builder.parser()
parser.SetAutoRenaming(True)

# Create two cubes using airo_models
cube1_size = (0.2, 0.2, 0.2)
cube2_size = (0.2, 0.2, 0.2)
cube1_urdf_path = airo_models.box_urdf_path(cube1_size, "cube1")
cube2_urdf_path = airo_models.box_urdf_path(cube2_size, "cube2")

# Add cubes to the scene
cube1_index = parser.AddModels(cube1_urdf_path)[0]
cube2_index = parser.AddModels(cube2_urdf_path)[0]

# Position the cubes in space
world_frame = plant.world_frame()
cube1_frame = plant.GetFrameByName("base_link", cube1_index)
cube2_frame = plant.GetFrameByName("base_link", cube2_index)

# Set transforms for the cubes
cube1_transform = RigidTransform(p=[0.5, 0.0, 0.3])  # Position cube1 at (0.5, 0, 0.3)
cube2_transform = RigidTransform(p=[-0.5, 0.0, 0.3])  # Position cube2 at (-0.5, 0, 0.6)

# Weld the cubes to the world frame at their positions
plant.WeldFrames(world_frame, cube1_frame, cube1_transform)
plant.WeldFrames(world_frame, cube2_frame, cube2_transform)

# Finish building the scene
robot_diagram, context = finish_build(robot_diagram_builder, meshcat)

# Force publish to visualize
robot_diagram.ForcedPublish(context)

# Visualize the frames of the cubes
visualize_frame(meshcat, "cube1_frame", cube1_transform, length=0.25)
visualize_frame(meshcat, "cube2_frame", cube2_transform, length=0.25)

INFO:drake:Meshcat listening for connections at http://localhost:7003
LCM self test failed!!
Check your routing tables and firewall settings


In [3]:
# Define goal state
goal_transform1 = RigidTransform(p=[-0.3, 0.4, 0.5])

In [4]:
class TwoCubeConstraint(ob.Constraint):
    def __init__(self, fixed_distance=1.0):
        # 6 dimensions (3D position for each cube)
        # 1 constraints: fixed distance only
        super(TwoCubeConstraint, self).__init__(6, 1)
        self.fixed_distance = fixed_distance
        
    def function(self, x, out):
        # Extract positions of both cubes
        p1 = x[:3]
        p2 = x[3:]
        
        # Constraint 1: Fixed distance between cubes
        dist = np.linalg.norm(p2 - p1)
        out[0] = dist - self.fixed_distance
        
        return True

In [5]:
# After setting start and goal states, but before solving:
# Check what distance we actually need
start_dist = np.linalg.norm(cube1_transform.translation() - cube2_transform.translation())
print(f"Start configuration distance: {start_dist}")

Start configuration distance: 1.0


In [6]:
@dataclass
class Options:
    space: str = "PJ"  # Projection-based state space
    planner: str = "RRTConnect"
    delta: float = 0.1
    lambda_: float = ob.CONSTRAINED_STATE_SPACE_LAMBDA
    tolerance: float = 1e-3
    time: float = 30.0
    tries: int = ob.CONSTRAINT_PROJECTION_MAX_ITERATIONS
    range: float = 0.0

# Setup OMPL planning
space = ob.RealVectorStateSpace(6)  # 6D state space (3D + 3D)

# Set bounds for the space
bounds = ob.RealVectorBounds(6)
for i in range(6):
    bounds.setLow(i, -2)  # Slightly larger than sphere radius
    bounds.setHigh(i, 2)
space.setBounds(bounds)

# Create constraint and problem
constraint = TwoCubeConstraint(fixed_distance=start_dist)
options = Options()
problem = ConstrainedProblem(options.space, space, constraint, options)

# Set start and goal states
start = ob.State(problem.css)
goal = ob.State(problem.css)

Info:    Using Projection-Based State Space!


In [7]:
# Set start state (current position of cubes)
start()[0] = cube1_transform.translation()[0]
start()[1] = cube1_transform.translation()[1]
start()[2] = cube1_transform.translation()[2]
start()[3] = cube2_transform.translation()[0]
start()[4] = cube2_transform.translation()[1]
start()[5] = cube2_transform.translation()[2]

# Set goal state
goal()[0] = goal_transform1.translation()[0]
goal()[1] = goal_transform1.translation()[1]
goal()[2] = goal_transform1.translation()[2]
goal()[5] = goal_transform1.translation()[2]

problem.setStartAndGoalStates(start, goal)
problem.setPlanner(options.planner)

# Solve the problem
def reset_visualization(meshcat):
    """Reset the visualization by clearing all frames except the cubes."""
    # Instead of meshcat.Delete(), we'll force publish the diagram again
    robot_diagram.ForcedPublish(context)
    
    # Re-visualize the initial cube frames
    visualize_frame(meshcat, "cube1_frame", cube1_transform, length=0.25)
    visualize_frame(meshcat, "cube2_frame", cube2_transform, length=0.25)

# Add this before your planning code
reset_visualization(meshcat)

solved = problem.solveOnce(output=True)

if solved:
    # Get the solution path
    path = problem.ss.getSolutionPath()
    path.interpolate()
    
    # Visualize the solution
    for i in range(path.getStateCount()):
        state = path.getState(i)
        
        # Create transforms for visualization
        cube1_transform_i = RigidTransform(p=[state[0], state[1], state[2]])
        cube2_transform_i = RigidTransform(p=[state[3], state[4], state[5]])
        
        # Visualize frames
        visualize_frame(meshcat, f"cube1_frame_{i}", cube1_transform_i, length=0.15)
        visualize_frame(meshcat, f"cube2_frame_{i}", cube2_transform_i, length=0.15)
        
        # Optional: add small delay to see motion
        import time
        time.sleep(0.1)
else:
    print("No solution found")

         at line 67 in /project/src/ompl/base/src/SpaceInformation.cpp


Debug:   RRTConnect: Planner range detected to be 1.959592
Info:    RRTConnect: Starting planning with 1 states already in datastructure
Info:    RRTConnect: Created 36663 states (36662 start + 1 goal)
Info:    ProblemDefinition: Adding approximate solution from planner RRTConnect
Info:    Solution found in 30.086073 seconds
Info:    Simplifying solution...
Info:    SimpleSetup: Path simplification took 0.001461 seconds and changed from 7 to 2 states
Info:    Simplified Path Length: 1.277 -> 1.277
Info:    Interpolating path...
Info:    Interpolating simplified path...
Info:    Dumping path to `ompl_path.txt`.
Info:    Dumping simplified path to `ompl_simplepath.txt`.


In [8]:
# Convert path to numpy array
n_states = path.getStateCount()
path_array = np.zeros((n_states, 6))  # 6 is the state dimension (3D + 3D)

for i in range(n_states):
    state = path.getState(i)
    path_array[i] = np.array([state[j] for j in range(6)])

In [9]:
# Compare last state with goal
last_state = path.getState(n_states - 1)
last_state_array = np.array([last_state[j] for j in range(6)])
print(f"Last state: {last_state_array}")
print(f"Goal: {goal_transform1.translation()}")



Last state: [-0.36265651  0.46595112  0.42181518  0.17686265 -0.36580251  0.55260114]
Goal: [-0.3  0.4  0.5]
